# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [2]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.csv')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Martin\AppData\Local\Temp\graphlab_server_1482177862.log.0


This non-commercial license of GraphLab Create for academic use is assigned to xinningh@gmail.com and will expire on October 24, 2017.


Finished parsing file C:\Users\Martin\kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.169153 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,float,long,float,long,long,float,long,long,long,long,long,long,long,long,long,float,float,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\Martin\kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.110097 secs.

In [7]:
sales['bathrooms'] = sales['bathrooms'].astype(float)

In [8]:
sales['waterfront'] = sales['waterfront'].astype(int)

In [9]:
sales['sqft_above'] = sales['sqft_above'].astype(int)

In [10]:
sales['sqft_living15'] = sales['sqft_living15'].astype(float)

In [27]:
sales['id'] = sales['id'].astype(str)    

In [12]:
sales['grade'] = sales['grade'].astype(int)

In [24]:
sales['sqft_basement'] = sales['sqft_basement'].astype(int)

In [25]:
sales['yr_built'] = sales['yr_built'].astype(int)

In [26]:
sales['sqft_lot'] = sales['sqft_lot'].astype(int)

In [23]:
sales['view'] = sales['view'].astype(int)

In [18]:
sales['sqft_living'] = sales['sqft_living'].astype(float)

In [19]:
sales['condition'] = sales['condition'].astype(int)

In [20]:
sales['lat'] = sales['lat'].astype(float)

In [21]:
sales['floors'] = sales['floors'].astype(str)

In [22]:
sales['date'] = sales['date'].astype(str)

In [13]:
sales['price'] = sales['price'].astype(float)

In [14]:
sales['bedrooms'] = sales['bedrooms'].astype(float)

In [15]:
sales['zipcode'] = sales['zipcode'].astype(str)

In [16]:
sales['long'] = sales['long'].astype(float)

In [17]:
sales['sqft_lot15'] = sales['sqft_lot15'].astype(float)

In [11]:
sales['yr_renovated'] = sales['yr_renovated'].astype(int)

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [4]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.014572     | 4146407.590959     | 258679.804259 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [6]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0692067  | 7873.33813675 |
| sqft_living |  None | 315.403441118  | 3.45570032294 |
|   bedrooms  |  None | -65080.2154416 | 2717.45685213 |
|  bathrooms  |  None | 6944.02087632  | 3923.11492813 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [7]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.504278


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [8]:
import numpy as np
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    predictions = np.array(predictions)
    # Then compute the residuals/errors
    actual = np.array(outcome)
    error = actual - predictions
    # Then square and add them up
    RSS = np.sum(error * error) 
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [9]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761534465e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [10]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [11]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [12]:
train_data_bed = np.array(train_data['bedrooms'])
train_data_bath = np.array(train_data['bathrooms'])
train_data['bed_bath_rooms'] = train_data_bed * train_data_bath
test_data_bed = np.array(test_data['bedrooms'])
test_data_bath = np.array(test_data['bathrooms'])
test_data['bed_bath_rooms'] = test_data_bed * test_data_bath

In [13]:
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

In [14]:
train_data_lat = np.array(train_data['lat'])
train_data_long = np.array(train_data['long'])
train_data['lat_plus_long'] = train_data_lat + train_data_long
test_data_lat = np.array(test_data['lat'])
test_data_long = np.array(test_data['long'])
test_data['lat_plus_long'] = test_data_lat + test_data_long

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [15]:
bedrooms_sq = np.array(test_data['bedrooms_squared'])
np.average(bedrooms_sq)

12.446677701584299

In [16]:
graphlab.canvas.set_target('ipynb')
test_data['bedrooms_squared'].show()

In [17]:
bed_bath_rooms = np.array(test_data['bed_bath_rooms'])
np.average(bed_bath_rooms)

7.5039016315913925

In [18]:
log_sqft_living = np.array(test_data['log_sqft_living'])
np.average(log_sqft_living)

7.5502746796459377

In [19]:
lat_plus_long = np.array(test_data['lat_plus_long'])
np.average(lat_plus_long)

-74.653333554031676

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [20]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [21]:
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, validation_set = None)
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, validation_set = None)
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.020029     | 3193116.484463     | 228199.723038 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [22]:
model1_weight_summary = model_1.get("coefficients")
print model1_weight_summary
model2_weight_summary = model_2.get("coefficients")
print model2_weight_summary
model3_weight_summary = model_3.get("coefficients")
print model3_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56141852.4692 | 1649928.35825 |
| sqft_living |  None | 310.263506724  | 3.18882062328 |
|   bedrooms  |  None | -59577.6052126 | 2487.27455846 |
|  bathrooms  |  None | 13812.4068763  | 3593.53440572 |
|     lat     |  None | 629863.119956  |  13120.68867  |
|     long    |  None | -214800.952332 | 13283.8203911 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]

+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -54412036.4015 | 1650346.09777 |
|  sqft_living   |  None |  304.44950524  | 3.20216518282 |
|    bedrooms    |  None | -116366.742819 | 4805.53398548 |
|   bathrooms    |  None | -77972.1056406 | 7565

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [23]:
rss_M1_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
print rss_M1_train
rss_M2_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
print rss_M2_train
rss_M3_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
print rss_M3_train

9.7132387161e+14
9.61587552577e+14
9.05273774726e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [24]:
rss_M1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
print rss_M1_test
rss_M2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
print rss_M2_test
rss_M3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print rss_M3_test

2.26566318597e+14
2.24366990581e+14
2.51824565049e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.